# Part I. Preparing a Docker Image

Before diving into the nitty-gritty of Sagemaker training and deploy, it is crutial to make sure the training and deploy "container" is set up. This container will provide the most up-to-date version of GluonCV, MXNet and other essential programming environments, which enable us to achieve state-of-the-art(SOTA) model training and deployment.
Let's take a look of the process of setting up a container.

### 1. Building a Docker Image

If this is the first time of using SageMaker training and deployment, you will need to prepare a Docker image by running the following commands:

In [19]:
%%writefile DockerfileNLP

ARG APP=sentiment_analysis    
    
# Use built-in DLC (Deep learning container) image
FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-training:1.6.0-gpu-py36-cu101-ubuntu16.04
RUN pip install --upgrade --user gluonnlp

# Set some environment variables. PYTHONUNBUFFERED keeps Python from buffering our standard
# output stream, which means that logs can be delivered to the user quickly. PYTHONDONTWRITEBYTECODE
# keeps Python from writing the .pyc files which are unnecessary in this case. We also update
# PATH so that the train and serve programs are found when the container is invoked.

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

# Set up the program in the image
COPY $APP /opt/program
WORKDIR /opt/program

Overwriting DockerfileNLP


In [20]:
%%writefile buildnlp.sh
#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The arguments to this script are the image name and application name
image=$1
app=$2
dockerfile=$3

chmod +x $app/train_sa
chmod +x $app/serve

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration
region=$(aws configure get region)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${image}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image}" > /dev/null
fi

# Edit ECR policy permission rights
aws ecr set-repository-policy --repository-name "${image}" --policy-text ecr_policy.json

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  -t ${image} --build-arg APP=$app . -f ${dockerfile}
docker tag ${image} ${fullname}

Overwriting buildnlp.sh


In [21]:
import os
# os.chdir("./container/") ## Change the working directory to `container`
os.getcwd()

!bash buildnlp.sh gluonnlp sentiment_analysis DockerfileNLP


An error occurred (InvalidParameterException) when calling the SetRepositoryPolicy operation: Invalid parameter at 'PolicyText' failed to satisfy constraint: 'Invalid repository policy provided'
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  96.26kB
Step 1/8 : ARG APP=sentiment_analysis
Step 2/8 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-training:1.6.0-gpu-py36-cu101-ubuntu16.04
 ---> b55e94690a48
Step 3/8 : RUN pip install --upgrade --user gluonnlp
 ---> Using cache
 ---> 94c62d3f2530
Step 4/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Running in 745b040aec1f
Removing intermediate container 745b040aec1f
 ---> 70554ab78ce4
Step 5/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Runn

### 2. Granting the ECR Repo Access

Since Amazon ECR repository policies are a subset of IAM policies that are scoped for, and specifically used for, controlling access to individual Amazon ECR repositories. IAM policies are generally used to apply permissions for the entire Amazon ECR service but can also be used to control access to specific resources as well. Amazon ECR requires that users have allow permissions to the ecr:GetAuthorizationToken API through an IAM policy before they can authenticate to a registry and push or pull any images from any Amazon ECR repository. More details: https://docs.aws.amazon.com/AmazonECR/latest/userguide/repository-policies.html

We can go to the [ECR Repo](https://console.aws.amazon.com/ecr/repositories/my-repo/permissions?region=us-east-1) to grant permission to access the repo with the permission like the following.

```{`json}
{
  "Version": "2008-10-17",
  "Statement": [
    {
      "Sid": "All-Allow",
      "Effect": "Allow",
      "Principal": {
        "AWS": [
          "arn:aws:iam::383827541835:user/rlhu",
          "arn:aws:sts::383827541835:assumed-role/AmazonSageMaker-ExecutionRole-20200409T103675/SageMaker"
        ]
      },
      "Action": "*"
    }
  ]
}
```

In [22]:
import sagemaker
sagemaker.get_execution_role()

'arn:aws:iam::383827541835:role/service-role/AmazonSageMaker-ExecutionRole-20200409T103675'

In [23]:
%%writefile push.sh
#!/usr/bin/env bash
image=$1
account=$(aws sts get-caller-identity --query Account --output text)
region=$(aws configure get region)
region=${region:-us-west-2}
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"
docker push ${fullname}

Overwriting push.sh


### 3. Pushing the Docker image

Now with the access permission to the new ECR repo, let's push Docker image by calling the `push.sh` script. 

In [24]:
!bash push.sh gluonnlp sentiment_analysis

The push refers to repository [383827541835.dkr.ecr.us-east-1.amazonaws.com/gluonnlp]

520992c1: Preparing 
f0a13406: Preparing 
30ca3f6c: Preparing 
29f12112: Preparing 
9888af4b: Preparing 
c74f8124: Preparing 
dc1ddd51: Preparing 
ff67b5d1: Preparing 
3f8452de: Preparing 
33b9b80c: Preparing 
e15c5efb: Preparing 
ada40cbc: Preparing 
98ec25b6: Preparing 
a135a886: Preparing 
cb108c96: Preparing 
8933565e: Preparing 
f67b5d1: Waiting g 
f8452de: Waiting g 
059844e3: Preparing 
3b9b80c: Waiting g 
2603e31f: Preparing 
15c5efb: Waiting g 
9333c486: Preparing 
80f86be3: Preparing 
8ec25b6: Waiting g 
7fbd6c92: Preparing 
9d138968: Preparing 


15c5efb: Pushing  222.7MB/1.725GBPushing  152.9MB/220.5MB

2603e31f: Pushing  408.7MB/2.293GBPushing  192.1MB/466.4MBPushing  161.9MB/2.293GB

2603e31f: Pushing  757.6MB/2.293GB

15c5efb: Pushing  1.223GB/1.725GBPushing  1.202GB/1.725GB

15c5efb: Pushing  1.579GB/1.725GB

2603e31f: Pushing  1.955GB/2.293GBPushing  1.802GB/2.293GB

2603e31f: Pushed   2.322GB/2.293GBlatest: digest: sha256:d92dd1a27e7dec072bca662464553644bbea6868f5dfc115a9773386fd5a8f2f size: 6181


# Part 2: Training, Infernce and Deployment


## 1. Training

Once we have the container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

### Setting up the Environment

Here we specify a bucket to use and the role that will be used for working with Amazon SageMaker.

In [25]:
os.chdir("../") ## Change the working directory back to main

from sagemaker import get_execution_role
role = get_execution_role()

### Creating the Session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our SageMaker operations.

In [26]:
import sagemaker as sage

sess = sage.Session()

### Defining the account, region and ECR address


In [27]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
ecr_name = "gluonnlp"
ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, ecr_name)

### Uploading Training Data

We can upload the training data to the corresponding S3 bucket: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-383827541835/sagemaker-deploy-gluoncv/data/?region=us-east-1

In [18]:
! ls ./data/minc-2500/train

brick  carpet  food  mirror  sky  water


In [19]:
# s3_bucket = "sagemaker-deploy-gluoncv"
# # model_path = "s3://{}/{}/model".format(sess.default_bucket(), s3_bucket)
# # os.path.join(model_path, "model.tar.gz")
# # model_prefix = s3_bucket + "/model"
# train_data_local = "./data/minc-2500/train"
# train_data_dir_prefix = s3_bucket + "/data/train"


# # model_local_path = "model_output"
# train_data_upload = sess.upload_data(path=train_data_local, 
# #                                 bucket=s3_bucket, 
#                                 key_prefix=train_data_dir_prefix)


Check the training data at : 

In [20]:
print("https://s3.console.aws.amazon.com/s3/buckets/{}".format(train_data_upload.split("//")[1]))

https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-383827541835/sagemaker-deploy-gluoncv/data/train


### Creating an Estimator

In order to use Amazon SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __role__ is defined as above.
* The __session__ is the SageMaker session object that we defined above.
* The __image name__ is the name of ECR image we created above.
* The __training instance type__ which is the type of machine to use for training.
* The __training instance count__ which is the number of machines to use for training.
* The __output path__ determines where the model artifact will be written.

In [30]:
from sagemaker.estimator import Estimator

s3_bucket = "kdd2020"
train_instance_type = 'ml.p2.xlarge' # 'ml.c4.2xlarge'  # 
s3_model_path = "s3://{}/{}/model".format(sess.default_bucket(), s3_bucket)
# model_path = os.path.join(s3_path, "model.tar.gz")
print(s3_model_path)

s3://sagemaker-us-east-1-383827541835/kdd2020/model


### Fitting the Estimator

Then we call the `fit()` function on the estimator to train against the data that we uploaded above.

In [32]:
classifier = Estimator(role=role, 
                       sagemaker_session=sess,
                       image_name=ecr_image, 
                       train_instance_count=1,
                       train_instance_type=train_instance_type,
#                        hyperparameters=hyperparameters,
#                        checkpoint_local_path="model_output/", 
                       output_path=s3_model_path
                       )
# train_data_upload = model_upload
classifier.fit('s3://sagemaker-us-east-1-383827541835/sagemaker-deploy-gluoncv/data/train')
# classifier.fit(train_data_upload)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-18 06:08:09 Starting - Starting the training job...
2020-08-18 06:08:11 Starting - Launching requested ML instances......
2020-08-18 06:09:32 Starting - Preparing the instances for training......
2020-08-18 06:10:31 Downloading - Downloading input data...
2020-08-18 06:11:07 Training - Downloading the training image..................
2020-08-18 06:14:04 Uploading - Uploading generated training model
2020-08-18 06:14:04 Failed - Training job failed
2020-08-18 06:13:57,376 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2020-08-18 06:13:57,402 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{}', 'SM_USER_ENTRY_POINT': '', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}', 'SM_INPUT_DATA_CONFIG': '{"training":{"RecordWrapperType":"None","S3DistributionType":"Fully

UnexpectedStatusException: Error for Training job gluonnlp-2020-08-18-06-08-09-013: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/sagemaker_training/trainer.py", line 85, in train
    entrypoint()
  File "/usr/local/lib/python3.6/site-packages/sagemaker_mxnet_container/training.py", line 93, in main
    train(environment.Environment())
  File "/usr/local/lib/python3.6/site-packages/sagemaker_mxnet_container/training.py", line 89, in train
    runner_type=runner_type)
  File "/usr/local/lib/python3.6/site-packages/sagemaker_training/entry_point.py", line 93, in run
    install(name=user_entry_point, path=environment.code_dir, capture_error=capture_error)
  File "/usr/local/lib/python3.6/site-packages/sagemaker_training/entry_point.py", line 118, in install
    entry_point_type = _entry_point_type.get(path, name)
  File "/usr/local/lib/python3.6/site-packages/sagemaker_training/_entry_point_type.py", li

## 2.Batch Inferencing

After our model has been trained, we simply use a demo image for testing our model. We first upload this image the S3 bucket and we can test the model after deplyment.

In [18]:
demo_dir = "data/demo"
test_image = "cat1.jpg"
sample_inference_input_prefix = s3_bucket + "/data/test"

demo_input = sess.upload_data(os.path.join(demo_dir, test_image), 
                                   key_prefix=sample_inference_input_prefix) 
print("Demo input uploaded to " + demo_input)

Demo input uploaded to s3://sagemaker-us-east-1-383827541835/sagemaker-deploy-gluoncv/data/test/cat1.jpg


##  3. Deploying the Model

Deploying the model to Amazon SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

Note that deployment takes a little bit longer than all the previous steps.

In [19]:
# from sagemaker.predictor import csv_serializer

model = classifier.create_model()
predictor = classifier.deploy(1, 'ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------!

### Choose some data and use it for a prediction

In order to do some predictions, we'll use a demo jpeg image to test the model.

In [20]:
with open(os.path.join(demo_dir, test_image), 'rb') as f:
    x = f.read()
    print(predictor.predict(x, initial_args={'ContentType':'image/jpeg'}).decode('utf-8'))

[lynx], with probability 0.253.
[Egyptian cat], with probability 0.252.
[tiger cat], with probability 0.106.
[tabby], with probability 0.063.
[soft-coated wheaten terrier], with probability 0.041.



### Cleanup Endpoint

When you're done with the endpoint, you'll want to clean it up.

In [ ]:
sess.delete_endpoint(predictor.endpoint)